In [1]:
import cv2
import numpy as np

In [2]:
#Dense optical flow method - the params taken by the function

# * prev first 8-bit single-channel input image.
# * next second input image of the same size and the same type as prev.
# * flow computed flow image that has the same size as prev and type CV_32FC2.
# * pyr_scale parameter, specifying the image scale (\<1) to build pyramids for each image
#     * pyr_scale=0.5 means a classical pyramid, where each next layer is twice smaller than the previous one.
    
# * levels number of pyramid layers including the initial image; levels=1 means that no extra layers are created and only the original images are used.
# * winsize averaging window size
#     * larger values increase the algorithm robustness to image
# * noise and give more chances for fast motion detection, but yield more blurred motion field.
# * iterations number of iterations the algorithm does at each pyramid level.
# * poly_n size of the pixel neighborhood used to find polynomial expansion in each pixel
#     * larger values mean that the image will be approximated with smoother surfaces, yielding more robust algorithm and more blurred motion field, typically poly_n =5 or 7.
# * poly_sigma standard deviation of the Gaussian that is used to smooth derivatives used as a basis for the polynomial expansion; for poly_n=5, you can set poly_sigma=1.1, for poly_n=7, a good value would be poly_sigma=1.5.

In [4]:
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()

prevImg = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:, :, 1] = 255
#make a hsv mask and the saturation is maxed out

while True:
    
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    #calc the OF
    flow = cv2.calcOpticalFlowFarneback(prevImg, nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    #return isa vector and sep to mag and angle
    
    mag, angle = cv2.cartToPolar(flow[:, :, 0], flow[:, :, 1], angleInDegrees = True) #x and y comp of flow vector fed in
    hsv_mask[:, :, 0] = angle / 2
    hsv_mask[:, :, 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX) #min pushed to 0 and max to 255bgr
    
    #conv the mask to bgr and show
    bgr = cv2.cvtColor(hsv_mask, cv2.COLOR_HSV2BGR)
    cv2.imshow('Detector', bgr)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        
    prevImg = nextImg
    
cap.release()
cv2.destroyAllWindows()

In [5]:
# #acc to the color space - 
# movements are like :
#     left = blue
#     right = red
#     up = purple
#     down = green
# this is based on the angle